In [67]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer

In [68]:
custom_stopwords = ['echo', 'alexa', "alexia", 'dot', "star", 'amazon', 'prime', '2nd', 'generation', "fire", "stick", "firestick", "skype", "facetime", '1st', '3rd', '4th', '5th', "hub", "hulu", 'google', 'netflix', "sony", 'youtube', 'philip', 'tp-link', 'fourth', 'roku', "siri", 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "...", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

def negation_handler(sentence):	

    """Handle negations using WordNet. See https://github.com/UtkarshRedd/Negation_handling for a clear explenation."""

    temp = int(0)
    for i in range(len(sentence)):
        if sentence[i-1] in ['not',"n't", "no", "without"]:
            antonyms = []
            for syn in wordnet.synsets(sentence[i]):
                syns = wordnet.synsets(sentence[i])
                w1 = syns[0].name()
                temp = 0
                for l in syn.lemmas():
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
                max_dissimilarity = 0
                for ant in antonyms:
                    syns = wordnet.synsets(ant)
                    w2 = syns[0].name()
                    syns = wordnet.synsets(sentence[i])
                    w1 = syns[0].name()
                    word1 = wordnet.synset(w1)
                    word2 = wordnet.synset(w2)
                    if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                        temp = 1 - word1.wup_similarity(word2)
                    if temp>max_dissimilarity:
                        max_dissimilarity = temp
                        antonym_max = ant
                        sentence[i] = antonym_max
                        sentence[i-1] = ''
    while '' in sentence:
        sentence.remove('')
    return sentence

In [69]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t")
dataset.drop(dataset[dataset.rating == 3].index, inplace=True) #droppa recensioni con 3 stelle
# dataset.drop(dataset[dataset.rating == 4].index, inplace=True) #droppa recensioni con 4 stelle
dataset.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [70]:
df_no_duplicates = dataset.drop_duplicates(subset="verified_reviews")
df_no_duplicates[df_no_duplicates["feedback"] == 1]
index_positive = list(dataset[dataset["feedback"] == 1].index)
dataset.drop(labels=index_positive, inplace = True)
dataset = pd.concat([dataset, df_no_duplicates])

In [71]:
print(dataset["feedback"].value_counts())

1    1990
0     463
Name: feedback, dtype: int64


In [72]:
#  CREAZIONE DI UN SAMPLE DATASET BILANCIATO
# prende le prime n recensioni positive di lunghezza maggiore, dove n è il numero di recensioni negative
def create_balanced_dataset(dataset):
    """Bilancia il dataset uniformando il numero di recensioni negative e positive. Prende in input il dataset"""
    reviews_1 = list(dataset[dataset["feedback"] == 1]["verified_reviews"])
    reviews_0 = list(dataset[dataset["feedback"] == 0]["verified_reviews"])
    reviews_1.sort(key=len, reverse = True)
    sample_1 = reviews_1[:len(reviews_0)]
    verified_reviews_sample = []
    feedback_sample = []
    verified_reviews_sample.extend(sample_1)
    verified_reviews_sample.extend(reviews_0)
    feedback_sample.extend([1 for i in range(len(sample_1))])
    feedback_sample.extend([0 for i in range(len(reviews_0))])
    dataset = pd.DataFrame({"verified_reviews":verified_reviews_sample, "feedback": feedback_sample})
    print(dataset["feedback"].value_counts())
    return dataset


# dataset = create_balanced_dataset(dataset)

In [73]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v).
    This was done in order to have compatibility with the wordnet lemmatizer.
    For example the pos "JJ" is transformed in "a".
    """
    if treebank_tag.startswith('J'):
        return "a"
    elif treebank_tag.startswith('V'):
        return "v"
    elif treebank_tag.startswith('N'):
        return "n"
    elif treebank_tag.startswith('R'):
        return "r"
    else:
        return "n"
        
pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
# pos_list = ["JJ", "JJR", "JJS", "RB", "RBR", "RBS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "NN", "NNS"]

def tokenize_list_of_text(list_of_text, custom_stopwords = [], pos_filter = False, pos_list = []):
    """Tokenizza tutte le recensioni, pulisce da stopwords, elimina token <= 2 caratteri e lemmatizza.
    Ritorna sia il la lista tokenizzata ma come stringa sia come lista di tokens, dunque ritorna due elementi"""

    tokenizer = nltk.tokenize.TweetTokenizer()
    lemmatizer = nltk.WordNetLemmatizer()
    detokenizer = TreebankWordDetokenizer()

    tokenized_reviews = []
    sent_tokenized_reviews = []
    pos_reviews = []
    for review in list_of_text: #pulisce le recensioni
        review = re.sub(r'\d+', '', review) # elimina i numeri
        clean_text = ""
        tokens = nltk.tokenize.word_tokenize(review, language='english', preserve_line=False)
        tokens = [w.lower() for w in tokens]
        tokens_pos = pos_tag(tokens)
        lemmatized_tokens = [(lemmatizer.lemmatize(w, get_wordnet_pos(pos)), pos) for w, pos in tokens_pos]
        if pos_filter:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords and pos in pos_list]
        else:
            clean_tokens = [w for w, pos in lemmatized_tokens if w not in string.punctuation and len(w)>2 and w not in custom_stopwords]
        clean_tokens = negation_handler(clean_tokens)
        sent_tokenized_reviews.append(clean_tokens)
        tokenized_reviews.append(detokenizer.detokenize(clean_tokens))
    
    n_tokens = []
    for sent in sent_tokenized_reviews:
        for w in sent:
            n_tokens.append(w)
    print("total number of tokens extracted are:", len(set(n_tokens)))
    return tokenized_reviews,  sent_tokenized_reviews # ritorna una tupla!

In [74]:
new_text, new_sent_tok = tokenize_list_of_text(dataset["verified_reviews"], custom_stopwords, False, pos_list)
# contengono una lista di tutte le frasi pre processate, nella prima variabile in stringa, nella seconda in tokens 

total number of tokens extracted are: 3174


In [75]:
negative_reviews = list(dataset[dataset["feedback"] == 0]["verified_reviews"].values)
negative_text, negative_tok = tokenize_list_of_text(negative_reviews, custom_stopwords, False, pos_list)

total number of tokens extracted are: 1240


In [76]:
w2v_model_neg = Word2Vec(negative_tok, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007, seed = 5)
w2v_model_neg.train(negative_tok, total_examples=len(negative_tok), epochs=100)

(732937, 871200)

In [77]:
def generate_samples(reviews = [], n = 100):
    
    import random
    from nltk.corpus import wordnet as wn

    random.seed(10)
    reviews_to_sample = random.sample(reviews, n)
    generated_reviews = []

    for review in reviews_to_sample:
        new_review = []
        for w in review:
            syns = wn.synsets(w, lang = "eng")
            synonyms = []
            for syn in syns:
                synonyms.extend(syn.lemma_names())
            sim_score = {}
            for word_sim in synonyms:
                if word_sim != w and word_sim != "" and "_" not in word_sim:
                    try:
                        score = w2v_model_neg.wv.similarity(w, word_sim)
                        sim_score[word_sim] = score
                    except KeyError:
                        continue
            
            if sim_score:
                new_review.append(max(sim_score, key=sim_score.get))
            else:
                new_review.append(w)

        if new_review:
            # print("-------------------------")
            # print(review)
            # print("-")
            # print(new_review)
            generated_reviews.append(new_review)

    return generated_reviews

In [78]:
negative_artifical = generate_samples(negative_tok, 150)
new_sent_tok.extend(negative_artifical)
len(negative_artifical)

141

In [79]:
negative_artifical

[['disconnection',
  'speaker',
  'clock',
  'day',
  'deal',
  'refurbish',
  'product',
  'attempt'],
 ['never',
  'reply',
  'talk',
  'promp',
  'news',
  'suppose',
  'would',
  'link',
  'upstairs',
  'function',
  'care',
  'intercom',
  'unable'],
 ['unhappy',
  'speaker',
  'heavy',
  'ground',
  'deal',
  'clock',
  'fun',
  'music',
  'depart',
  'brand',
  'weird',
  'interference',
  'contain',
  'son',
  'family',
  'practically',
  'sound'],
 ['contain',
  'run',
  'month',
  'guarantee',
  'sound',
  'month',
  'would',
  'take',
  'product',
  'deal',
  'would',
  'high',
  'standard',
  'warranty',
  'would',
  'run',
  'unit',
  'month',
  'think',
  'product',
  'think',
  'bit',
  'dust',
  'crack',
  'dismiss',
  'lack',
  'purchase',
  'another',
  'one',
  'would',
  'lack',
  'purchase',
  'another',
  'one'],
 ['non',
  'crack',
  'move',
  'sink',
  'would',
  'tightness',
  'anything',
  'stupid',
  'plenty',
  'mark',
  'among',
  'spotify',
  'history',
  

In [80]:
def frequency_cleaning(new_sent_tok, n):

    tot_tokens = []

    for sent in new_sent_tok:
        for tok in sent:
            tot_tokens.append(tok)

    freqs = nltk.FreqDist(tot_tokens)
    cleaned_reviews = []

    for sent in new_sent_tok:
        clean_sent = []
        for tok in sent:
            if freqs[tok] > n:
                clean_sent.append(tok)
        cleaned_reviews.append(clean_sent)

    return cleaned_reviews

In [81]:
# Elimino token con freq minore di 5
cleaned_reviews = frequency_cleaning(new_sent_tok, 4)
negative_artificial_cleaned = cleaned_reviews[-len(negative_artifical):]
del cleaned_reviews[-len(negative_artifical):]

In [82]:
bigrams = Phrases(cleaned_reviews) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][10]

['item',
 'longer',
 'work',
 'month',
 'use',
 'unplug',
 'wifi',
 'unresponsive',
 'reset',
 'request']

In [83]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], list(dataset["feedback"].values), test_size=0.20, random_state=10)
X_train.extend(negative_artificial_cleaned)
Y_train.extend([0 for x in range(len(negative_artifical))])

In [122]:
# Questa parte estrare le parole più rilevanti per ogni categoria, ma non la uso alla fine.
# Lo riprenderò più avanti
# FARE IL K-SQUARED

neg_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 0]
pos_tokenized = [rev for rev, feedback in zip(X_train, Y_train) if feedback == 1]
neg_tokenized = [w for rev in neg_tokenized for w in rev]
pos_tokenized = [w for rev in pos_tokenized for w in rev]

import math 

def get_pos_neg_score(pos, neg):

    from sklearn.preprocessing import MinMaxScaler

    total_tokens = pos + neg
    total_tokens_freq = nltk.FreqDist(total_tokens)
    pos_freq = nltk.FreqDist(pos)
    neg_freq = nltk.FreqDist(neg)
    data_dict = {"pos": {}, "neg": {}}

    for w in pos:
        p_category = pos_freq[w] / total_tokens_freq[w]
        p = pos_freq[w] / len(pos)
        data_dict["pos"][w] = p_category * p
    
    for w in neg:
        p_category = neg_freq[w] / total_tokens_freq[w]
        p = neg_freq[w] / len(neg)
        data_dict["neg"][w] = p_category * p

    scaler = MinMaxScaler()
    scaled_pos = scaler.fit_transform(list(data_dict["pos"].values()))
    scaled_neg = scaler.fit_transform(list(data_dict["neg"].values()))

    data_dict["pos"] = {k: v for k, v in zip(data_dict["pos"], scaled_pos)}
    data_dict["neg"] = {k: v for k, v in zip(data_dict["neg"], scaled_neg)}
    
    pos_sorted = {k: v for k, v in sorted(data_dict["pos"].items(), key = lambda item: item[1], reverse=True)}
    neg_sorted = {k: v for k, v in sorted(data_dict["neg"].items(), key = lambda item: item[1], reverse=True)}
    data_dict["pos"] = pos_sorted
    data_dict["neg"] = neg_sorted

    return data_dict

data_dict = get_pos_neg_score(pos_tokenized, neg_tokenized)

ValueError: Expected 2D array, got 1D array instead:
array=[4.77421922e-04 2.26304836e-03 2.17009964e-04 2.01887156e-03
 1.00821222e-02 3.44804721e-03 2.65234401e-03 6.30897361e-03
 4.13153586e-04 2.01608914e-02 2.98388701e-04 3.09135267e-03
 4.59059540e-05 3.88117326e-02 4.24375041e-04 3.03125030e-04
 1.38111342e-03 3.48992633e-04 7.71590984e-05 9.72526137e-04
 9.78342681e-04 8.28857503e-04 1.65771501e-05 1.85664081e-03
 3.61013490e-04 5.14393990e-04 1.02304697e-03 2.48347811e-03
 3.31543001e-04 1.32617200e-04 7.85233424e-04 3.01402728e-04
 4.33704507e-03 2.45511843e-03 3.31543001e-04 3.11919240e-03
 3.36212621e-03 2.65392279e-03 2.54992390e-02 3.05019561e-03
 4.64160202e-04 1.49194351e-04 1.76822934e-04 1.02013231e-03
 1.91631855e-03 1.22010111e-02 2.37728611e-03 1.56020236e-05
 1.31702599e-03 1.51906975e-03 1.19355480e-04 5.22307743e-03
 9.76504653e-03 7.29394602e-04 4.71527824e-04 1.54567634e-03
 9.28320403e-04 1.25986340e-03 2.74771939e-03 3.52727307e-03
 8.86124021e-04 1.32617200

In [120]:
data_dict["neg"]

[0.009673925634605968,
 0.00786365448715125,
 0.006621516184031125,
 0.004997940256298918,
 0.004796076809170098,
 0.004695304695304696,
 0.004133926483850496,
 0.003988433113886754,
 0.003980416746374193,
 0.003951367781155015,
 0.0038275690485183125,
 0.0035234434046903127,
 0.0035234434046903127,
 0.003293653617427901,
 0.003277219234666043,
 0.0032733224222585926,
 0.0031715024851606865,
 0.003156418050035072,
 0.002919757979436236,
 0.002840776245031564,
 0.002840776245031564,
 0.002822405557967868,
 0.0028102012553731046,
 0.0026720999365376264,
 0.0025718961889174656,
 0.0025209378728200333,
 0.002519490780679335,
 0.00250720153632775,
 0.00250261952389612,
 0.0024720167252701855,
 0.0023823880486814417,
 0.002290459737268248,
 0.002290459737268248,
 0.0022632686462473696,
 0.0022586807010733143,
 0.002242718088182814,
 0.0022267499471146884,
 0.0021059978819678443,
 0.002078616764657242,
 0.0020295897955472423,
 0.0020291258893082604,
 0.0019935271894958345,
 0.0019935271894958

In [85]:
# tfidf weighting sui token, per ora senza pos. Mi sono reso conto che la pos per averla più accurata la devo fare prima del pre processing.

tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 12
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

able 0.2249249015304568
bedroom 0.24097804450421667
completely 0.33130002262354297
dark 0.3253626203370814
display 0.28769579953237184
easy 0.1978179622227001
favorite 0.28511012221747173
love 0.12106485427700273
music 0.17379176912088873
night 0.2583613347239237
skill 0.25547999875620503
spot 0.22569452954549868
try 0.20985334058742994
turn 0.2280765312063493
use 0.1389891369758205
wake 0.2583613347239237
yet 0.2501688114270335


In [86]:
w2v_model = Word2Vec(X_train, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007, seed = 5)

In [87]:
w2v_model.train(X_train, total_examples=len(X_train), epochs=100)

(1921430, 2363500)

In [88]:
w2v_model.wv.most_similar("love", topn = 10)

[('great', 0.25955426692962646),
 ('blast', 0.22110874950885773),
 ('security_camera', 0.21915099024772644),
 ('travel', 0.21872034668922424),
 ('like', 0.20897917449474335),
 ('improvement', 0.20838351547718048),
 ('guy', 0.20754249393939972),
 ('absolutely_love', 0.20073740184307098),
 ('outstanding', 0.19625253975391388),
 ('listen', 0.1946718841791153)]

In [89]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('point', -0.03687885403633118),
 ('research', -0.038739051669836044),
 ('shortly', -0.03875153511762619),
 ('loose', -0.04134296253323555),
 ('switch', -0.04256429150700569),
 ('source', -0.045874983072280884),
 ('solution', -0.04850295931100845),
 ('previous', -0.04925130307674408),
 ('characteristic', -0.055115025490522385),
 ('assume', -0.07657280564308167)]

In [90]:
df = pd.DataFrame(tfidf_model.toarray(), columns = tfidf.get_feature_names_out())
df

,'ll,'re,'ve,....,.....,......,ability,able,absolutely,absolutely_love,...,yard,year,year_old,yell,yes,yet,yield,z-wave,zero,zigbee
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.298178,0.0
2100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [91]:
g_model = []
# google_model = api.load('word2vec-google-news-300', True)
# g_model = KeyedVectors.load_word2vec_format(google_model, binary=True)

In [105]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False, test = False):

    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * peso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    if test:
        for word in tokens:
            try:
                if pretrained:
                    vec += g_model[word] * weight
                else:
                    vec += w2v_model.wv[word] * weight
                count +=1
            except KeyError:
                print("non trovo", word)
                continue

        if count!= 0:
            vec = vec / norm(vec)
        
        return vec

    if len(tokens) != len(weights):
        print("nope")

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += g_model[word] * weight
            else:
                vec += w2v_model.wv[word] * weight
            count +=1
        except KeyError:
            print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [106]:
# qui viene creato per ogni recensione il vettore

w2v_X_train = np.zeros((len(X_train), 300))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], 300, tfidf_model[i,:].data, False)
w2v_df = pd.DataFrame(w2v_X_train)
w2v_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.037436,0.024968,0.109724,0.018178,0.082140,0.041093,-0.019538,0.019814,-0.007271,-0.107329,...,-0.037278,0.026202,0.067330,0.017581,0.005319,-0.000364,0.014661,0.089783,0.076761,-0.055050
1,-0.058679,-0.103617,-0.081639,0.074817,0.011864,-0.001366,-0.010609,0.008731,0.018614,0.029657,...,-0.012424,0.019532,0.081359,0.070816,0.034008,0.028387,-0.135459,0.072594,0.043265,-0.056014
2,-0.037436,0.024968,0.109724,0.018178,0.082140,0.041093,-0.019538,0.019814,-0.007271,-0.107329,...,-0.037278,0.026202,0.067330,0.017581,0.005319,-0.000364,0.014661,0.089783,0.076761,-0.055050
3,0.027533,0.021382,0.023901,0.103499,0.074559,0.028210,-0.081313,-0.008650,-0.088448,0.039854,...,0.011349,0.045512,0.066154,0.027690,0.040464,0.008225,-0.022252,0.074263,0.148502,-0.050379
4,-0.029783,-0.014268,0.085866,0.041078,0.038151,0.006630,0.034895,0.016084,-0.048982,-0.034830,...,-0.004770,-0.027468,0.075778,0.040369,-0.132114,-0.042492,-0.024968,0.010592,0.081344,-0.039806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,-0.063894,0.128551,-0.016874,0.079798,-0.042847,0.069412,-0.085770,0.042757,-0.067661,0.048353,...,0.001023,-0.029823,0.114998,0.048743,-0.115133,0.058256,-0.088071,-0.015225,-0.013587,-0.108568
2099,-0.042294,0.006577,0.009470,0.067912,-0.086389,0.072339,-0.040169,0.002420,-0.029050,-0.006668,...,0.071058,0.027249,0.094250,-0.000389,-0.128791,0.038306,-0.065405,0.047628,0.112749,0.108817
2100,-0.003261,-0.057798,-0.012224,0.090025,-0.089591,0.070062,-0.078967,0.041580,-0.035892,-0.029071,...,-0.068082,-0.068496,0.049755,0.012977,-0.085023,-0.012781,-0.061325,0.154539,0.070906,-0.032483
2101,-0.011849,0.050466,-0.038193,0.036060,0.039839,0.016367,-0.068111,0.007635,-0.055993,-0.030789,...,-0.037224,-0.005990,0.074731,0.037903,0.027928,-0.049965,-0.084224,0.102279,0.076535,-0.035102


In [107]:
X_test_vec = np.zeros((len(X_test), 300))
for i in range(len(X_test)):
    X_test_vec[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], 300, X_test_tf[i,:].data, False)

X_test_vec[0]

array([-9.05013369e-02,  3.18331525e-02,  1.03953749e-04,  7.04872237e-02,
       -3.75421942e-02,  9.59559557e-02, -8.49666755e-02,  2.62024787e-02,
        4.08488146e-02, -7.77732907e-02, -8.29825782e-02,  6.39312411e-02,
       -3.85351705e-02, -9.85354431e-03, -5.60825939e-02, -7.34746177e-03,
        2.67223191e-02, -4.68593325e-02,  5.29977439e-02, -6.61357467e-02,
        9.87989856e-02,  7.01824383e-03, -2.12569635e-02, -5.96846640e-02,
        2.93789424e-02, -3.17789459e-02, -8.63818565e-02,  2.72658673e-02,
        1.00540260e-02,  3.64937733e-02,  4.96376616e-02, -2.97857677e-03,
       -7.31932823e-02, -8.63674473e-02,  1.17384774e-02, -2.19364326e-02,
       -6.72807633e-02,  5.73805180e-02,  4.67527008e-02,  9.97386124e-03,
       -4.56938607e-02, -1.59690151e-01,  1.10843650e-02, -1.03785154e-01,
       -2.72021819e-03,  2.59736979e-02, -1.74989699e-03, -7.82405835e-03,
       -1.43918794e-02, -4.99542362e-02, -5.00860720e-03,  7.22378863e-02,
       -2.21282360e-02,  

In [108]:
svm = SVC(max_iter=3000, class_weight = "balanced", C = 100, gamma = 1, kernel = "rbf")
svm_model = svm.fit(w2v_X_train, Y_train)
predictions = svm_model.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90        85
           1       0.99      0.97      0.98       406

    accuracy                           0.96       491
   macro avg       0.92      0.96      0.94       491
weighted avg       0.97      0.96      0.96       491



In [29]:
# risultati migliori:

# w2v + tf-idf weights + pos/neg score + svm and word count > 4

# w2v configuration:
# w2v_model = Word2Vec(cleaned_reviews, vector_size=300, window = 5, min_count = 0, sg=1, hs = 1, alpha=0.03, min_alpha=0.0007) and 100 epochs for training

# otherwise the balanced dataset performs better

In [30]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [31]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear', 'poly'],
              'class_weight':['balanced', None]
}

In [32]:
grid = GridSearchCV(SVC(), param_grid, refit = True, scoring = "f1_macro", cv=kfold, verbose=3)
grid.fit(w2v_X_train, Y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.876 total time=   0.5s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.851 total time=   0.4s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.851 total time=   0.4s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.892 total time=   0.4s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=rbf;, score=0.895 total time=   0.5s
[CV 1/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.857 total time=   0.2s
[CV 2/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.833 total time=   0.2s
[CV 3/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.817 total time=   0.2s
[CV 4/5] END C=0.1, class_weight=balanced, gamma=1, kernel=linear;, score=0.876 total time=   0.2s
[CV 5/5] END C=0.1, class_weight=balanced, gamma=1, kernel=lin

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': ['balanced', None],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             scoring='f1_macro', verbose=3)

In [33]:
print(grid.best_params_)

{'C': 10, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'rbf'}


In [34]:
grid_predictions = grid.predict(X_test_vec)

print(classification_report(Y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89        85
           1       0.98      0.97      0.98       406

    accuracy                           0.96       491
   macro avg       0.92      0.94      0.93       491
weighted avg       0.96      0.96      0.96       491



In [66]:
svm = SVC(max_iter=3000, class_weight = "balanced", C = 100, gamma = 1, kernel = "rbf")
svm_model = svm.fit(w2v_X_train, Y_train)
predictions = svm_model.predict(X_test_vec)
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88       138
           1       0.98      0.96      0.97       598

    accuracy                           0.95       736
   macro avg       0.91      0.93      0.92       736
weighted avg       0.95      0.95      0.95       736

